<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

# RNA in PyRosetta
Keywords: classify_base_pairs, RNA torsions, RNA score terms, RNA motifs, mutate_position, RNA thread

In this lab, we will explore common tasks and approaches for working with RNA using Rosetta. We will be focusing on a simple system that includes a helix capped by a tetraloop for this exercise.

In [1]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

In [2]:
from pyrosetta import *
init()

PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python37.mac 2019.35+release.767c1ea25c572fbc078db45ad65dddb507240ad3 2019-08-22T09:19:33] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python37.mac r231 2019.35+release.767c1ea25c5 767c1ea25c572fbc078db45ad65dddb507240ad3 http://www.pyrosetta.org 2019-08-22T09:19:33
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/ramyar/Dropbox/GradSchool/Research/packages/PyRosetta4.Release.python37.mac.release-231/setup/build/lib/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=621303715 seed_offset=0 real_seed=621303715
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=621303715 RG_type=mt19937


In [3]:
from pyrosetta.rosetta import *
from pyrosetta.rosetta.core.pose.rna import *
from pyrosetta.rosetta.core.pose import *

## Exploring geometry for RNA ##

Let's load in this structure with PyRosetta (make sure that you have the PDB file located in your current directory):

`cd google_drive/My\ Drive/student-notebooks/
pose = pose_from_pdb("inputs/stem_loop.pdb")`

In [4]:
### BEGIN SOLUTION
pose = pose_from_pdb("inputs/stem_loop.pdb")
### END SOLUTION

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 980 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.978188 seconds.
core.import_pose.import_pose: File 'inputs/stem_loop.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.chirality_resolution: Flipping atom xyz for H5' and H5'' for residue   C


Let's explore the structure in this PDB file. First, use `pose.sequence()` to look at the sequence:

In [5]:
# print out the sequence of the pose
### BEGIN SOLUTION
pose.sequence()
### END SOLUTION

'cauccgaaaggaug'

We can see that the pose seems to contain RNA residues. To check this, let's go through the pose residue by residue, checking if each one is RNA.

In [6]:
for ii in range(pose.size()):
    print(pose.residue_type(5).is_RNA())

True
True
True
True
True
True
True
True
True
True
True
True
True
True


RNA bases interact with each other via **base pairing**, either through the Watson-Crick base pairs that make up standard A-form helices or through non-canonical base pairing interactions. We can use the `classify_base_pairs` function (this lives in `core:pose:rna` which was loaded above) to find and classify all the base pairs in the current pose. Let's take a look.

In [7]:
base_pairs = classify_base_pairs(pose)
for base_pair in base_pairs:
    print(base_pair)

basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
1 14 WC WC ANTI
2 13 WC WC ANTI
3 12 WC WC ANTI
4 11 WC WC ANTI
5 10 WC WC ANTI
6 9 SUGAR HOOG ANTI


We can see that the RNA molecule consists of Watson-Crick base pairs between residues 1-5 and residues 10-14 forming a standard RNA helix. We can also see that residues 6 and 9 form a non-canonical base pair interaction between the sugar and Hoogsteen edges of the respective bases. We can think of this structure as a simple stem-loop, with an idealized A-form helix between residues 1-5 and residues 10-14, and with a tetraloop connecting these chains.

Let's use some of Rosetta's tools for measuring **distances and torsions** to understand the typical geometry of an idealized A-form helix.

What is the distance between the phosphate atoms of two consecutive residues in one strand of a helix? Check this for a couple pairs of residues.

In [8]:
P1_xyz = pose.residue(1).xyz("P")
P2_xyz = pose.residue(2).xyz("P")
P3_xyz = pose.residue(3).xyz("P")
print((P1_xyz - P2_xyz).norm())
print((P2_xyz - P3_xyz).norm())

5.923353746008057
5.846964169549871


RNA nucleotides are quite large compared to amino acids, with many more torsion angles. In the diagram of a nucleotide below, we can see the backbone torsions applicable to RNA: $\alpha$, $\beta$, $\gamma$, $\delta$, $\epsilon$, $\zeta$, and $\chi$.

![nucleotide_torsions](Media/nucleotide_torsions.png)

We can access the values of these torsions through the pose object. Just like protein torsions can be accessed with functions like `pose.phi(resid)`, RNA torsions can be accessed with analogous functions like `pose.alpha(resid)`.

**Exercise**: Below, make a function that prints out all the torsions for a given residue. Then, using that function, check the torsions for three different residues in the RNA helix. How similar are torsion angles for different residues in an idealized helix?

In [9]:
def print_torsions(pose, resi):
    print("%s: %f" % ("alpha", pose.alpha(resi)))
    print("%s: %f" % ("beta", pose.beta(resi)))
    print("%s: %f" % ("gamma", pose.gamma(resi)))
    print("%s: %f" % ("delta", pose.delta(resi)))
    print("%s: %f" % ("epsilon", pose.epsilon(resi)))
    print("%s: %f" % ("zeta", pose.zeta(resi)))
    print("%s: %f" % ("chi", pose.chi(resi)))

In [10]:
print("Torsions for residue 2:")
print_torsions(pose, 2)
print("Torsions for residue 3:")
print_torsions(pose, 3)
print("Torsions for residue 12:")
print_torsions(pose, 12)

Torsions for residue 2:
alpha: -72.516450
beta: 178.544461
gamma: 58.327244
delta: 79.813633
epsilon: -152.375244
zeta: -70.561778
chi: 74.780524
Torsions for residue 3:
alpha: -65.899308
beta: 173.462004
gamma: 56.858161
delta: 79.609718
epsilon: -152.592269
zeta: -70.540365
chi: 77.981938
Torsions for residue 12:
alpha: -67.502850
beta: 174.408225
gamma: 56.171977
delta: 81.812182
epsilon: -149.905716
zeta: -76.028946
chi: 75.291052


## Scoring RNA poses ##

Rosetta's energy functions provide a mechanism to score RNA structures, rewarding realistic conformations using a variety of score terms. In this section, we will see how to score RNA poses, and we will use these score terms to better understand our structure.

To score structures with RNA in Rosetta, it is best to use a high-resolution energy function designed to work with RNA, for instance `stepwise/rna/rna_res_level_energy4.wts`. In fact, the standard high resolution energy function used in Rosetta does not include the score terms that are quite helpful for modeling RNA. To see this, we will evaluate our RNA pose with the `ref2015` score function and `stepwise/rna/rna_res_level_energy4.wts`, comparing the resulting score term values.

In [11]:
hires_sf = core.scoring.ScoreFunctionFactory.create_score_function("ref2015");

core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/rama/fd/all.ramaProb
basic.io.database: Database file opened: scoring/score_functions/rama/fd/prepro.ramaProb
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.all.txt
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.gly.txt
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.pro.txt
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.valile.txt
basic.io.database: Database file opened: scoring/score_functions/P_AA_pp/P_AA
basic.io.database: Dat

In [12]:
hires_sf.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -146.195    -146.195
 fa_rep                       0.550      21.661      11.914
 fa_sol                       1.000     181.903     181.903
 fa_intra_rep                 0.005     104.969       0.525
 fa_intra_sol_xover4          1.000      43.757      43.757
 lk_ball_wtd                  1.000     -23.168     -23.168
 fa_elec                      1.000     -11.254     -11.254
 pro_close                    1.250       0.000       0.000
 hbond_sr_bb                  1.000       0.000       0.000
 hbond_lr_bb                  1.000       0.000       0.000
 hbond_bb_sc                  1.000       0.000       0.000
 hbond_sc                     1.000     -23.799     -23.799
 dslf_fa13                    1.250       0.000       0.000
 omega  

Note that `ref2015` does contain some terms that are used for RNA modeling like VDW and hydrogen bonding score terms. What extra terms are included in the RNA high resolution score function?

In [13]:
rna_hires_sf = core.scoring.ScoreFunctionFactory.create_score_function("stepwise/rna/rna_res_level_energy4.wts");

core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.


In [14]:
rna_hires_sf.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       0.210    -212.399     -44.604
 fa_rep                       0.200      57.744      11.549
 fa_intra_rep                 0.003     186.211       0.540
 lk_nonpolar                  0.250      -9.222      -2.305
 fa_elec_rna_phos_phos        1.700       0.378       0.643
 rna_torsion                  1.000       3.751       3.751
 suiteness_bonus              1.000       0.000       0.000
 rna_sugar_close              0.820       3.938       3.229
 fa_stack                     0.130    -234.442     -30.477
 stack_elec                   0.760      -0.903      -0.686
 geom_sol_fast                0.170      74.843      12.723
 hbond_sr_bb_sc               0.960       0.000       0.000
 hbond_lr_bb_sc               0.960       0.000       0.000
 hbond_s

We can see that some new score terms in the high resolution RNA potential, including `rna_torsion`, `suiteness_bonus`, `rna_sugar_close`, and `fa_stack`. We will explore a few of these terms below. To learn about these and other score terms that have been included to more realistically model RNA, check out these papers: 

Analogous to the protein low-resolution potential, an RNA low-resolution potential has been developed to more quickly score RNA structures represented in centroid mode. Lets take a look at the score terms involved.

In [15]:
rna_lowres_sf = core.scoring.ScoreFunctionFactory.create_score_function("rna/denovo/rna_lores_with_rnp_aug.wts");

basic.io.database: Database file opened: scoring/rna/rna_atom_vdw.txt
basic.io.database: Database file opened: scoring/rna/rnp_atom_vdw_min_distances_reformat_MIN.txt
core.scoring.ScoringManager: Reading in: /Users/ramyar/Dropbox/GradSchool/Research/packages/PyRosetta4.Release.python37.mac.release-231/setup/build/lib/pyrosetta/database/scoring/rna/rna_base_pair_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Reading basepair x - y potential file: scoring/rna/rna_base_pair_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Finished reading basepair x - y potential file: scoring/rna/rna_base_pair_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Reading non - base - base x - y potential file: scoring/rna/rna_base_backbone_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Reading RNA backbone backbone potential file: scoring/rna/rna_backbone_backbone.dat
core.scoring.rna.RNP_LowResPotential: Reading RNP basepair x - y potential file: scoring/rna/rnp_base_pair.txt
core.scoring.rna

In [16]:
rna_lowres_sf.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 rna_data_backbone            2.000       0.000       0.000
 rna_vdw                      2.000       0.221       0.442
 rnp_vdw                     50.000       0.000       0.000
 rna_base_backbone            2.000      -1.884      -3.769
 rna_backbone_backbone        2.000       0.000       0.000
 rna_repulsive                5.000       0.000       0.000
 rna_base_pair                2.000     -53.214    -106.429
 rna_base_axis                0.400     -32.234     -12.894
 rna_base_stagger             2.000     -28.072     -56.143
 rna_base_stack               2.000      -3.243      -6.486
 rna_base_stack_axis          0.400     -20.598      -8.239
 rnp_base_pair                2.000       0.000       0.000
 rnp_stack                    5.000       0.000       0.000
 rnp_pai

We can see that when modeling an RNA molecule using centroid positions for nucleotides, we need to separately include terms for base pairing (`rna_base_pair`), base-backbone interactions (`rna_base_backbone`), and so on. 

Returning to the high resolution RNA score function, let us see if we can decompose the energies further to understand which parts of the structure contribute positively and negatively to its energy. First, we can decompose the energies per residue like below.

In [17]:
rna_hires_sf(pose)
nonzero_scores = pose.energies().residue_total_energies(4).show_nonzero()
print(nonzero_scores)

 fa_atr: -16.0481 fa_rep: 4.95774 fa_intra_atr: -10.6601 fa_intra_rep: 10.4003 fa_intra_sol: 6.2042 fa_intra_atr_nonprotein: -10.6601 fa_intra_rep_nonprotein: 10.4003 fa_intra_sol_nonprotein: 6.2042 lk_nonpolar: -0.837546 fa_elec_rna_phos_phos: -0.0122409 rna_torsion: 0.105802 rna_torsion_sc: 0.054655 rna_sugar_close: 0.471854 fa_stack: -14.7643 stack_elec: 0.0989445 geom_sol_fast: 6.69203 geom_sol_fast_intra_RNA: 0.348984 hbond_sc: -2.22045 ref: 2.82 total_score: -2.10402


A lot of the RNA specific energy terms make more sense when we look at pairs of residues. The energy graph object allows you to explore pairwise energies. The function below uses the energy graph to print out all non-zero scores between residues for a particular score term.

In [18]:
def print_nonzero_pairwise_energies(pose, energy_term, sf):
    sf(pose)
    energy_graph = pose.energies().energy_graph()
    for ii in range(1, pose.size() + 1):
        for jj in range(ii + 1, pose.size() + 1):
            edge = energy_graph.find_energy_edge(ii, jj)
            if (edge != None):
                emap = edge.fill_energy_map()
                resid1 = str(ii) + " " + pose.residue(ii).name1()
                resid2 = str(jj) + " " + pose.residue(jj).name1()
                resid_pair = resid1 + " " + resid2
                score = emap[ energy_term ]
                if score != 0:
                    print("%s: %f" % (resid_pair, score))

Using the function above, we're going to look at the stacking energies in the high resolution potential.

In [19]:
print_nonzero_pairwise_energies(pose, core.scoring.ScoreType.fa_stack, rna_hires_sf)

1 c 2 a: -8.786457
1 c 13 u: -0.021402
1 c 14 g: -0.014325
2 a 3 u: -17.195315
2 a 12 a: -1.166573
2 a 13 u: -0.021531
2 a 14 g: -18.136998
3 u 4 c: -10.651433
3 u 11 g: -0.182032
3 u 12 a: -0.025817
3 u 13 u: -2.437063
4 c 5 c: -11.016306
4 c 6 g: -0.042999
4 c 10 g: -0.170933
4 c 11 g: -0.003559
4 c 12 a: -7.643363
5 c 6 g: -14.998700
5 c 9 a: -0.055888
5 c 10 g: -0.004817
5 c 11 g: -9.170599
6 g 8 a: -1.056694
6 g 9 a: -0.381851
6 g 10 g: -11.820445
7 a 8 a: -21.526270
8 a 9 a: -22.045880
9 a 10 g: -13.441684
10 g 11 g: -18.409410
11 g 12 a: -18.811699
12 a 13 u: -16.199935
13 u 14 g: -9.002159


We can see that the **stacking energies** are highest for consecutive residues. In the idealized helix, the best stacking energy bonuses are given to stacked purines.

Now lets take a look at the **torsion energies**. Which energies are the highest? Where are these torsions in the structure?

In [20]:
print_nonzero_pairwise_energies(pose, core.scoring.ScoreType.rna_torsion, rna_hires_sf)

1 c 2 a: 0.041018
2 a 3 u: 0.003446
3 u 4 c: 0.012547
4 c 5 c: 0.015482
5 c 6 g: 0.048322
6 g 7 a: 0.273854
7 a 8 a: 0.096358
8 a 9 a: 0.020082
9 a 10 g: 0.248952
10 g 11 g: 0.019879
11 g 12 a: 0.012364
12 a 13 u: 0.017076
13 u 14 g: 0.002773


RNA structures are often viewed as being composed of small building blocks called **RNA motifs**. These motifs can be as simple as stacks of base pairs, which we have seen above. Typical motifs also include stereotyped loops, junctions, and tertiary contacts present across many common RNA molecules. Let's take a look to see whether any of these common RNA motifs are present in our simple stem loop structure.

In [21]:
lowres_potential = core.scoring.rna.RNA_LowResolutionPotential( "scoring/rna/rna_base_pair_xy.dat" )
rna_scoring_info = core.scoring.rna.rna_scoring_info_from_pose(pose).rna_filtered_base_base_info()
rna_motifs = core.scoring.rna.get_rna_motifs( pose, lowres_potential, rna_scoring_info)
print(rna_motifs)

core.scoring.rna.RNA_LowResolutionPotential: Reading basepair x - y potential file: scoring/rna/rna_base_pair_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Finished reading basepair x - y potential file: scoring/rna/rna_base_pair_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Reading non - base - base x - y potential file: scoring/rna/rna_base_backbone_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Reading RNA backbone backbone potential file: scoring/rna/rna_backbone_backbone.dat
WC_STACKED_PAIR [1, 2, 13, 14]
WC_STACKED_PAIR [4, 5, 10, 11]
WC_STACKED_PAIR [11, 12, 3, 4]
WC_STACKED_PAIR [10, 11, 4, 5]
WC_STACKED_PAIR [3, 4, 11, 12]
WC_STACKED_PAIR [12, 13, 2, 3]
WC_STACKED_PAIR [2, 3, 12, 13]
WC_STACKED_PAIR [13, 14, 1, 2]
U_TURN [6, 7, 8]
GNRA_TETRALOOP [6, 7, 8, 9]



We can see that our RNA structure includes many stacked Watson-Crick base pair, making the idealized A-form helix. In addition, the loop connecting the strands of the helix in our structure is a stereotyped "GNRA" tetraloop, taking a loop conformation that is common across many RNA structures in the PDB.

## Manipulating RNA poses ##

Rosetta allows you to not just explore a given PDB structure, but to manipulate and design structures. In this section, we discuss some basic ways to manipulate RNA structures, and we observe the effects of these manipulations on the structure's energy. For each manipulation, we will make a new copy of the pose to make sure that our changes do not affect the original structure we loaded in.

One basic manipulation we can make to an RNA structure is to change torsion angles for individual residues. Let's try this out on a residue in the A-form helix, and observe the effect on the rna_torsion score. Did the change we made make the score better or worse?

In [22]:
new_pose = Pose()
new_pose.assign(pose)
rna_hires_sf(pose)
torsion_score_before = pose.energies().total_energies()[core.scoring.ScoreType.rna_torsion]
new_pose.set_beta(2, 110)
rna_hires_sf(new_pose)
torsion_score_after = new_pose.energies().total_energies()[core.scoring.ScoreType.rna_torsion]
print("%s: %f" % ("Torsion score before", torsion_score_before))
print("%s: %f" % ("Torsion score after", torsion_score_after))

Torsion score before: 3.750853
Torsion score after: 4.166565


If you want to replace residues in an RNA molecule with their idealized versions, you can use the RNA_IdealCoord class in Rosetta. Below is an example for using that method to first replace a single residue with its idealized version, and then to replace all residues with their idealized versions across the whole pose.

In [23]:
ideal_pose_one = Pose()
ideal_pose_one.assign(pose)
resid = 2
core.pose.rna.RNA_IdealCoord().apply(ideal_pose_one, resid, core.chemical.rna.PuckerState.ANY_PUCKER, False)

ideal_pose = Pose()
ideal_pose.assign(pose)
core.pose.rna.RNA_IdealCoord().apply(ideal_pose, False)

**Exercise**: Figure out if the total energy of the pose went up or down after replacing one or all of the residues with their idealized versions. What can explain the difference?

In [24]:
rna_hires_sf(pose)
rna_hires_sf(ideal_pose)
rna_hires_sf(ideal_pose_one)
print(pose.energies().total_energy())
print(ideal_pose.energies().total_energy())
print(ideal_pose_one.energies().total_energy())

-18.045673691089746
22.63480325590245
-1.1025695607212


Another common manipulation for an RNA structure is to mutate the nucleotides to different bases. This is a manipulation that is commonly used while modeling one RNA structure using coordinates from another homologous (but not identical) structure. Below we can see how to mutate one residue of our RNA structure to another one. We include the `update_full_model_info_from_pose` command to ensure that the pose's internal data is kept up-to-date after the mutation is made.

In [30]:
mutated_pose = Pose()
mutated_pose.assign(pose)
print(pose.sequence())
rosetta.core.pose.rna.mutate_position(mutated_pose, 1, 'a')
core.pose.full_model_info.update_full_model_info_from_pose(mutated_pose)
print(mutated_pose.sequence())

cauccgaaaggaug
aauccgaaaggaug


**Exercise**: Make a function that mimics the 'rna_thread' Rosetta application, which takes in a pose and a new sequence and replaces all pose residues with the new sequence's residues. Remember to check that the pose's sequence and the new sequence have the same length. 

The pose's current sequence is `cauccgaaaggaug`. Use the function you wrote to make a version that has sequence `cauccuucgggaug` and one that has sequence `aaaaagaaauuuuu`.

In [34]:
def rna_thread(pose, new_seq):
    if len(pose.sequence()) != len(new_seq):
        print("Sequences have different length; cannot rethread")
        return pose
    for ii in range(1, pose.size()+1):
        rosetta.core.pose.rna.mutate_position(pose, ii, new_seq[ii-1])
        core.pose.full_model_info.update_full_model_info_from_pose(pose)
    return pose   

In [35]:
uucg_pose = Pose()
uucg_pose.assign(pose)
uucg_pose = rna_thread(uucg_pose, 'cauccuucgggaug')
print(uucg_pose.sequence())

auhelix_pose = Pose()
auhelix_pose.assign(pose)
auhelix_pose = rna_thread(auhelix_pose, 'aaaaagaaauuuuu')
print(auhelix_pose.sequence())

cauccuucgggaug
aaaaagaaauuuuu


**Exercise**: The RNA high resolution potential includes hydrogen bonding terms. CG base pairs have more hydrogen bonds than AU base pairs. Compare the original pose with the pose that has all AU base pairs. What happens to the hydrogen bonding energy in the high resolution potential? 

In [37]:
rna_hires_sf(auhelix_pose)
rna_hires_sf(pose)
auhelix_hbond_sc = auhelix_pose.energies().total_energies()[core.scoring.ScoreType.hbond_sc]
pose_hbond_sc = pose.energies().total_energies()[core.scoring.ScoreType.hbond_sc]
print('%s: %f' % ("AU Helix", auhelix_hbond_sc))
print('%s: %f' % ("CG Helix", pose_hbond_sc))

AU Helix: -18.166948
CG Helix: -23.799493


**Exercise**: The stacking energies of the GAAA and UUCG tetraloops differ from each other. Which tetraloop provides the most favorable stacking energies overall? Can you figure out which pairs of residues have different stacking energies when the structure has changed? 

In [38]:
rna_hires_sf(uucg_pose)
rna_hires_sf(pose)
uucg_fa_stack = uucg_pose.energies().total_energies()[core.scoring.ScoreType.fa_stack]
pose_fa_stack = pose.energies().total_energies()[core.scoring.ScoreType.fa_stack]
print('%s: %f' % ("UUCG Loop", uucg_fa_stack))
print('%s: %f' % ("GAAA Loop", pose_fa_stack))

UUCG Loop: -214.060807
GAAA Loop: -234.442137


In [39]:
rna_hires_sf(uucg_pose)
rna_hires_sf(pose)
pose_energy_graph = pose.energies().energy_graph()
uucg_energy_graph = uucg_pose.energies().energy_graph()
for ii in range(1, pose.size() + 1):
    for jj in range(ii + 1, pose.size() + 1):
        pose_edge = pose_energy_graph.find_energy_edge(ii, jj)
        uucg_edge = uucg_energy_graph.find_energy_edge(ii, jj)
        if (pose_edge != None)  and (uucg_edge != None):
            pose_emap = pose_edge.fill_energy_map()
            uucg_emap = uucg_edge.fill_energy_map()
            resid1 = str(ii) + " " + pose.residue(ii).name1()
            resid1_uucg = str(ii) + " " + uucg_pose.residue(ii).name1()
            resid2 = str(jj) + " " + pose.residue(jj).name1() 
            resid2_uucg = str(jj) + " " + uucg_pose.residue(jj).name1()
            resid_pair = resid1 + ", " + resid2
            resid_pair_uucg = resid1_uucg + ", " + resid2_uucg
            pose_score = pose_emap[ core.scoring.ScoreType.fa_stack ]
            uucg_score = uucg_emap[ core.scoring.ScoreType.fa_stack ]
            if pose_score != uucg_score:
                print("%s: %f; %s: %f" % (resid_pair, pose_score, resid_pair_uucg, uucg_score))

4 c, 6 g: -0.042999; 4 c, 6 u: -0.020146
5 c, 6 g: -14.998700; 5 c, 6 u: -14.424303
5 c, 9 a: -0.055888; 5 c, 9 g: -0.040952
6 g, 8 a: -1.056694; 6 u, 8 c: -0.798130
6 g, 9 a: -0.381851; 6 u, 9 g: -0.142689
6 g, 10 g: -11.820445; 6 u, 10 g: -5.138552
7 a, 8 a: -21.526270; 7 u, 8 c: -14.881668
8 a, 9 a: -22.045880; 8 c, 9 g: -16.479345
9 a, 10 g: -13.441684; 9 g, 10 g: -13.063297
